## Técnica de Regresión Polinómica

In [1]:
import time
import pandas as pd
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, max_error, r2_score
from sklearn.preprocessing import OneHotEncoder

# Cargar datos desde el archivo CSV
df_datos = pd.read_csv("datos/insurance.csv")

# Codificación One Hot para la columna 'region'
codificador_one_hot = OneHotEncoder(sparse=False)
codificacion_one_hot = codificador_one_hot.fit_transform(df_datos[['region']])
arr_nombre_nuevas_columnas = ['region_' + str(cat) for cat in codificador_one_hot.categories_[0]]
df_nuevas_columnas_one_hot = pd.DataFrame(codificacion_one_hot, columns=arr_nombre_nuevas_columnas)
df_datos = pd.concat([df_datos, df_nuevas_columnas_one_hot], axis=1)

# Columnas incluyendo 'region'
columnas_especificas = ['age', 'bmi'] + arr_nombre_nuevas_columnas + ['children', 'sex']

# Crear conjuntos de datos de entrenamiento y prueba para las columnas seleccionadas
df_X = pd.DataFrame(df_datos, columns=columnas_especificas)
df_y = pd.DataFrame(df_datos, columns=['charges'])
df_X_train, df_X_test, df_y_train, df_y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=100)

# Medir tiempo de preprocesamiento
start_time_preprocesamiento = time.time()

# Lista para almacenar los resultados
resultados_polinomicos = []

# Grados de regresión polinómica
grados_polinomicos = [2, 4, 6]

# Iterar sobre los grados polinómicos
for grado in grados_polinomicos:

    # Conjunto para rastrear combinaciones únicas
    combinaciones_unicas = set()

    # Iterar sobre el rango de columnas desde 1 hasta n
    for r in range(1, len(columnas_especificas) + 1):
        # Generar combinaciones de tamaño r
        combinaciones_columnas = combinations(columnas_especificas, r)

        # Iterar sobre cada combinación de columnas
        for combo in combinaciones_columnas:
            # Verificar si la combinación ya ha sido procesada
            combinacion_ordenada = tuple(sorted(combo))
            if combinacion_ordenada in combinaciones_unicas:
                continue

            # Agregar la combinación a las combinaciones únicas
            combinaciones_unicas.add(combinacion_ordenada)

            # Crear conjuntos de datos de entrenamiento y prueba para las columnas seleccionadas
            arr_X_train_subset = df_X_train[list(combo)].copy()
            arr_X_test_subset = df_X_test[list(combo)].copy()

            # Llenar valores faltantes con el valor medio de la respectiva columna
            arr_X_train_subset.fillna(arr_X_train_subset.mean(), inplace=True)
            arr_X_test_subset.fillna(arr_X_test_subset.mean(), inplace=True)

            # Eliminar filas con valores faltantes en el conjunto de entrenamiento
            indices_train_subset = arr_X_train_subset.dropna().index
            arr_X_train_subset = arr_X_train_subset.loc[indices_train_subset].to_numpy()
            arr_y_train_subset = df_y_train.loc[indices_train_subset].to_numpy()

            # Eliminar filas con valores faltantes en el conjunto de prueba
            indices_test_subset = arr_X_test_subset.dropna().index
            arr_X_test_subset = arr_X_test_subset.loc[indices_test_subset].to_numpy()
            arr_y_test_subset = df_y_test.loc[indices_test_subset].to_numpy()

            # Verificar si hay suficientes muestras para entrenar el modelo
            if len(arr_X_train_subset) == 0 or len(arr_X_test_subset) == 0:
                print(f"No hay suficientes muestras para la combinación {', '.join(combo)}")
                continue

            # Crear y entrenar el modelo de regresión polinómica
            model = make_pipeline(PolynomialFeatures(degree=grado), LinearRegression())
            model.fit(arr_X_train_subset, arr_y_train_subset)

            # Predicciones
            arr_y_test_predicho = model.predict(arr_X_test_subset)

            # Métricas de evaluación del modelo
            max_ea = max_error(arr_y_test_subset, arr_y_test_predicho)
            mae = mean_absolute_error(arr_y_test_subset, arr_y_test_predicho)
            mse = mean_squared_error(arr_y_test_subset, arr_y_test_predicho)
            r2 = r2_score(arr_y_test_subset, arr_y_test_predicho)

            # Almacenar resultados en la lista
            resultados_polinomicos.append({
                'Grado_Polinómico': grado,
                'Combinación_Columnas': ', '.join(combo),
                'Máximo_EA': max_ea,
                'EA_Medio': mae,
                'R^2': r2
            })

# Crear un DataFrame a partir de la lista de resultados
df_resultados_polinomicos = pd.DataFrame(resultados_polinomicos)
#df_resultados_knn = df_resultados_knn.sort_values(by='EA_Medio')
# Imprimir el tiempo de preprocesamiento
print(f"Tiempo de preprocesamiento polinómico: {time.time() - start_time_preprocesamiento} segundos")
print('***************************************************************************************')
# Ordenar el DataFrame por índice en orden ascendente
df_resultados_polinomicos = df_resultados_polinomicos.sort_index()

# Imprimir resultados
for idx, row in df_resultados_polinomicos.iterrows():
    print(f"{idx + 1:2d}. Grado Polinómico: {row['Grado_Polinómico']}, Combinación: {row['Combinación_Columnas']}")
    print(f"   Máximo EA: {row['Máximo_EA']:.2f}")
    print(f"   EA Medio: {row['EA_Medio']:.2f}")
    print(f"   R^2: {row['R^2']:.4f}")
    print("-" * 90)

C:\Users\rimel\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Tiempo de preprocesamiento polinómico: 11.646993160247803 segundos
***************************************************************************************
 1. Grado Polinómico: 2, Combinación: age
   Máximo EA: 48072.50
   EA Medio: 9472.05
   R^2: 0.0243
------------------------------------------------------------------------------------------
 2. Grado Polinómico: 2, Combinación: bmi
   Máximo EA: 45345.47
   EA Medio: 9062.06
   R^2: 0.0500
------------------------------------------------------------------------------------------
 3. Grado Polinómico: 2, Combinación: region_northeast
   Máximo EA: 46956.50
   EA Medio: 9093.82
   R^2: -0.0067
------------------------------------------------------------------------------------------
 4. Grado Polinómico: 2, Combinación: region_northwest
   Máximo EA: 47824.50
   EA Medio: 9120.18
   R^2: -0.0072
------------------------------------------------------------------------------------------
 5. Grado Polinómico: 2, Combinación: region_sout